# Del 15: Procesiranje velikih datasetov - hitrost

In [ ]:
import pandas as pd
import numpy as np

## CPU Bound Programs

### Bounds vs Limitations

<img alt="I/O bounds" src="images/CPU+and+I_O+bounds.png">

### Primer optimizacije

In [ ]:
import numpy as np

# Define a basic Haversine distance formula
def haversine(lat1, lon1, lat2, lon2):
    MILES = 3959
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    total_miles = MILES * c
    return total_miles

In [ ]:
df = pd.read_csv('data/new_york_hotels.csv')

#### Crude looping over DataFrame rows using indices

In [ ]:
# Define a function to manually loop over all rows and return a series of distances
def haversine_looping(df):
    distance_list = []
    for i in range(0, len(df)):
        d = haversine(40.671, -73.985, df.iloc[i]['latitude'], df.iloc[i]['longitude'])
        distance_list.append(d)
    return distance_list

In [ ]:
%%timeit
# Run the haversine looping function
df['distance'] = haversine_looping(df)

#### Looping with iterrows()

In [ ]:
%%timeit
# Haversine applied on rows via iteration
haversine_series = []
for index, row in df.iterrows():
    haversine_series.append(haversine(40.671, -73.985, row['latitude'], row['longitude']))
df['distance'] = haversine_series

#### Looping with apply()

In [ ]:
%%timeit

# Timing apply on the Haversine function
df['distance'] = df.apply(lambda row: haversine(40.671, -73.985, row['latitude'], row['longitude']), axis=1)

#### Vectorization with Pandas series

In [ ]:
%%timeit 
# Vectorized implementation of Haversine applied on Pandas series
df['distance'] = haversine(40.671, -73.985, df['latitude'], df['longitude'])

####  Vectorization with NumPy arrays

In [ ]:
%%timeit
# Vectorized implementation of Haversine applied on NumPy arrays
df['distance'] = haversine(40.671, -73.985, df['latitude'].values, df['longitude'].values)

This brings us to a few basic conclusions on optimizing Pandas code:
1. Avoid loops; they’re slow and, in most common use cases, unnecessary.
2. If you must loop, use apply(), not iteration functions.
3. Vectorization is usually better than scalar operations. Most common operations in Pandas can be vectorized.
4. Vector operations on NumPy arrays are more efficient than on native Pandas series.

## I/O Bound Programs

### I/O Bounds

<img src="./images/report_assembly.png">

<img src="./images/report_assembly_bidir.png">

## Optimizing Python Code with pandas

### Basic Looping

### Select columns and rows efficiently


In [ ]:
data = pd.read_csv('data/school.csv')
data.head(3)

In [ ]:
data['City'].value_counts().head(10)

In [ ]:
# save the top cities in a list
top_cities = ['Brooklyn','Bronx','Manhattan','Jamaica','Long Island City']

In [ ]:
%%timeit
data.loc[(data['City'].isin(top_cities) == False),'City'] = 'Others'

In [ ]:
data.City.value_counts()

In [ ]:
data = pd.read_csv('data/school.csv')

In [ ]:
%%timeit
data['City'][(data['City'].isin(top_cities) == False)] = 'Others'
# salba praksa

In [ ]:
data.City.value_counts()

### Uporaba biult-in funkciji

### Joining on indexes is faster than joining on columns

In [ ]:
n = 100000

i1 = np.arange(n)
np.random.shuffle(i1)
df1 = pd.DataFrame({'i': i1,
                    'j': np.random.randint(1,1000,n),
                    'k': np.random.randint(1,1000,n)})

i2 = np.arange(n)
np.random.shuffle(i1)
df2 = pd.DataFrame({'i': i2,
                    'm': np.random.randint(1,1000,n),
                    'n': np.random.randint(1,1000,n)})

In [ ]:
df1.head()

In [ ]:
df2.head()

In [ ]:
%%timeit
df1.merge(df2, on="i")

In [ ]:
df1 = df1.set_index('i')
df2 = df2.set_index('i')

In [ ]:
%%timeit
df1.merge(df2, left_index=True, right_index=True)

## PRIMER: Pohitritev pandas kode

### Naloga

**The goal of this example will be to apply time-of-use energy tariffs to find the total cost of energy consumption for one year.**



### Priprava podatkov

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('data/demand_profile.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.dtypes

In [ ]:
type(df.iloc[0, 0])

In [ ]:
df['date_time'] = pd.to_datetime(df['date_time'])
df['date_time'].dtype

In [ ]:
df.head()

In [ ]:
def convert(df, column_name):
    return pd.to_datetime(df[column_name])

df = pd.read_csv('data/demand_profile.csv')
df_coverted = df.copy()

In [ ]:
%%timeit -r 3 -n 10
df_coverted['date_time'] = convert(df, 'date_time')

In [ ]:
def convert_with_format(df, column_name):
    return pd.to_datetime(df[column_name], format='%d/%m/%y %H:%M')

In [ ]:
%%timeit -r 3 -n 10
df_coverted['date_time'] = convert_with_format(df, 'date_time')

In [ ]:
859/18

In [ ]:
df_coverted.head()

In [ ]:
df_coverted.info()

### 1) Simple Looping Over Pandas Data

<table class="table table-hover">
<thead>
<tr>
<th>Tariff Type</th>
<th>Cents per kWh</th>
<th>Time Range</th>
</tr>
</thead>
<tbody>
<tr>
<td>Peak</td>
<td>28</td>
<td>17:00 to 24:00</td>
</tr>
<tr>
<td>Shoulder</td>
<td>20</td>
<td>7:00 to 17:00</td>
</tr>
<tr>
<td>Off-Peak</td>
<td>12</td>
<td>0:00 to 7:00</td>
</tr>
</tbody>
</table>

In [ ]:
df_test = df_coverted.copy()
df_test['cost_cents'] = df['energy_kwh'] * 28

In [ ]:
df_test.head()

In [ ]:
def apply_tariff(kwh, hour):
    """Calculates cost of electricity for given hour."""    
    if 0 <= hour < 7:
        rate = 12
    elif 7 <= hour < 17:
        rate = 20
    elif 17 <= hour < 24:
        rate = 28
    else:
        raise ValueError(f'Invalid hour: {hour}')
    return rate * kwh

In [ ]:
# NOTE: Don't do this!
def apply_tariff_loop(df):
    """Calculate costs in loop.  Modifies `df` inplace."""
    energy_cost_list = []
    for i in range(len(df)):
        # Get electricity used and hour of day
        energy_used = df.iloc[i]['energy_kwh']
        hour = df.iloc[i]['date_time'].hour
        energy_cost = apply_tariff(energy_used, hour)
        energy_cost_list.append(energy_cost)
    df['cost_cents'] = energy_cost_list

In [ ]:
%%timeit -r 3 -n 10
apply_tariff_loop(df_coverted)

### 2) Looping with .itertuples() and .iterrows()

In [ ]:
for index, row in df[:5].iterrows():
    print(index)
    print(row)
    print('energy_kwh' ,row['energy_kwh'])
    print('-------')

In [ ]:
def apply_tariff_iterrows(df):
    energy_cost_list = []
    for index, row in df.iterrows():
        # Get electricity used and hour of day
        energy_used = row['energy_kwh']
        hour = row['date_time'].hour
        # Append cost list
        energy_cost = apply_tariff(energy_used, hour)
        energy_cost_list.append(energy_cost)
    df['cost_cents'] = energy_cost_list

In [ ]:
%%timeit -r 3 -n 10
apply_tariff_iterrows(df_coverted)

### 3) Pandas’ .apply()

In [ ]:
def apply_tariff_withapply(df):
    df['cost_cents'] = df.apply(lambda row: apply_tariff(
        kwh=row['energy_kwh'],
        hour=row['date_time'].hour), axis=1)

In [ ]:
%%timeit -r 3 -n 10
apply_tariff_withapply(df_coverted)

### 4) Selecting Data With .isin()

In [ ]:
df_coverted = df.copy()
df_coverted['date_time'] = convert_with_format(df, 'date_time')
df_coverted.set_index('date_time', inplace=True)

In [ ]:
df_coverted.head()

In [ ]:
def apply_tariff_isin(df):
    # Define hour range Boolean arrays
    peak_hours = df.index.hour.isin(range(17, 24))
    shoulder_hours = df.index.hour.isin(range(7, 17))
    off_peak_hours = df.index.hour.isin(range(0, 7))
    
    # Apply tariffs to hour ranges
    df.loc[peak_hours, 'cost_cents'] = df.loc[peak_hours, 'energy_kwh'] * 28
    df.loc[shoulder_hours,'cost_cents'] = df.loc[shoulder_hours, 'energy_kwh'] * 20
    df.loc[off_peak_hours,'cost_cents'] = df.loc[off_peak_hours, 'energy_kwh'] * 12

In [ ]:
%%timeit -r 3 -n 10
apply_tariff_isin(df_coverted)

In [ ]:
203/6.5

### 5) Pandas’ pd.cut() function

In [ ]:
df_coverted = df.copy()
df_coverted['date_time'] = convert_with_format(df, 'date_time')
df_coverted.set_index('date_time', inplace=True)

In [ ]:
pd.cut(x=df_coverted.index.hour,
      bins=[0,7,17,24],
      include_lowest=True,
      labels=[12, 20, 28])

In [ ]:
def apply_tariff_cut(df):
    cents_per_kwh = pd.cut(x=df_coverted.index.hour,
      bins=[0,7,17,24],
      include_lowest=True,
      labels=[12, 20, 28]).astype(int)
    df['cost_cents'] = cents_per_kwh * df['energy_kwh']

In [ ]:
%%timeit -r 3 -n 10
apply_tariff_cut(df_coverted)

### 6) Using NumPy

In [ ]:
import numpy as np

In [ ]:
df_coverted = df.copy()
df_coverted['date_time'] = convert_with_format(df, 'date_time')
df_coverted.set_index('date_time', inplace=True)

In [ ]:
np.digitize(df_coverted.index.hour.values, bins=[7, 17, 24])

In [ ]:
def apply_tariff_digitize(df):
    prices = np.array([12, 20, 28])
    bins = np.digitize(df.index.hour.values, bins=[7, 17, 24])
    df['cost_cents'] = prices[bins] * df['energy_kwh'].values

In [ ]:
%%timeit -r 3 -n 10
apply_tariff_digitize(df_coverted)

### Prevent Reprocessing with HDFStore

[What is HDF5](https://portal.hdfgroup.org/display/knowledge/What+is+HDF5)

In [ ]:
df_coverted.info()

In [ ]:
# Create storage object with filename `processed_data`
data_store = pd.HDFStore('data/OUT_processed_data.h5')

# Put DataFrame into the object setting the key as 'preprocessed_df'
data_store['preprocessed_df'] = df_coverted
data_store.close()

In [ ]:
data_store = pd.HDFStore('data/OUT_processed_data.h5')

# Retrieve data using key
preprocessed_df = data_store['preprocessed_df']
data_store.close()

In [ ]:
preprocessed_df.info()

## Drugi nasveti

### [Dask](https://www.dask.org/)

https://docs.dask.org/en/stable/install.html

###  [Numba](https://numba.pydata.org/)

Numba translates Python functions to optimized machine code at runtime using the industry-standard LLVM compiler library. Numba-compiled numerical algorithms in Python can approach the speeds of C or FORTRAN.

### pandas.eval() for Efficient Operations

[Dokumentacija](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.eval.html)

[High-Performance Pandas: eval() and query()](https://jakevdp.github.io/PythonDataScienceHandbook/03.12-performance-eval-and-query.html#pandas.eval()-for-Efficient-Operations)